<h1>Word2Vecf on Chess Puzzle Data</h1>

In [2]:
#Word 2 Vec F test
import numpy as np
import pandas as pd

<h2>Run Word2Vecf </h2>

In [2]:
#Run Word2VecF and store vector file in plain text 'dim200puzzlevec'
import subprocess
file_data = "word2vecf/train_data"
file_wv = "word2vecf/wv"
file_cv = "word2vecf/cv"
output = "word2vecf/vectors/dim200puzzlevec"
command = f"./word2vecf -train {file_data} -wvocab {file_wv} -cvocab {file_cv} -output {output} -threads 50 -alpha 0.26 -size 200"
output = subprocess.run(command,capture_output=True,shell=True).stdout
print(output)

b'Starting training using file train_data\nVocab size: 35362\nWord count: 5630373\nVocab size: 75165\nWord count: 5630373\nthread 0 69\nthread 5 40016793\nthread 6 48020153\nthread 9 72030216\nthread 4 32013420\nthread 7 56023496\nthread 2 16006751\nthread 3 24010095\nthread 10 80033547\nthread 8 64026844\nthread 12 96040279\nthread 1 8003366\nthread 11 88036889\nthread 20 160067042\nthread 14 112046956\nthread 16 128053660\nthread 13 104043599\nthread 26 208087187\nthread 25 200083833\nthread 18 144060375\nthread 22 176073798\nthread 33 264110665\nthread 17 136056985\nthread 21 168070443\nthread 15 120050333\nthread 32 256107288\nthread 19 152063748\nthread 23 184077152\nthread 27 216090528\nthread 36 288120735\nthread 31 248103981\nthread 28 224093860\nthread 29 232097263\nthread 35 280117385\nthread 37 296124077\nthread 41 328137468\nthread 40 320134138\nthread 44 352147525\nthread 24 192080491\nthread 45 360150893\nthread 34 272113989\nthread 46 368154194\nthread 43 344144138\n\rAl

<h2>Load Generated Embeddings</h2>

In [149]:
vectors = pd.read_csv("word2vecf/vectors/dim200puzzlevec", sep=' ', header=None, skiprows=1)
#Remove Empty column at end of file
vectors = vectors.drop(vectors.shape[1]-1,axis=1)
#Create puzzle_vocabulary data frame from first column of vectors and then remove vector id column from vectors
vectors.rename(columns={0:'tactics_problem_id'}, inplace=True)
puzzle_vocabulary_pd = vectors['tactics_problem_id'].to_frame().astype('int32').set_index("tactics_problem_id")
vectors = vectors.drop('tactics_problem_id',axis=1)

In [137]:
vectors.shape

(35362, 200)

<h2>Load Puzzle MetaData</h2>

In [139]:
#Load Fen Data
fen_data = pd.read_csv('/w/225/1/chess/tactics/tactics_problem.csv')

In [140]:
#Extract PGN and FEN
'''
import re
old_pgn = fen_data['pgn']
new_pgn = np.array([])
for i in range(0,len(old_pgn)):
    old_fen = old_pgn[i]
    try:
        string = re.search('\[FEN \"(.+?)\"\]', old_fen).group(1)
        string = string.replace(' ','_')
        #print(string)
    except:
        string = ''
    new_pgn = np.append(new_pgn,'https://lichess.org/analysis/' + string)
print(new_pgn)
'''

'\nimport re\nold_pgn = fen_data[\'pgn\']\nnew_pgn = np.array([])\nfor i in range(0,len(old_pgn)):\n    old_fen = old_pgn[i]\n    try:\n        string = re.search(\'\\[FEN "(.+?)"\\]\', old_fen).group(1)\n        string = string.replace(\' \',\'_\')\n        #print(string)\n    except:\n        string = \'\'\n    new_pgn = np.append(new_pgn,\'https://lichess.org/analysis/\' + string)\nprint(new_pgn)\n'

In [142]:
#np.save('fen_links.npy',new_pgn)
#Add FEN Data to 
new_pgn = np.load('fen_links.npy')
fen_data['pgn'] = new_pgn
fen_data.rename(columns={'pgn':'fen'}, inplace=True)

In [143]:
#puzzle_vocabulary_pd = puzzle_vocabulary_pd.join(tag_data.set_index('tactics_problem_id'))
puzzle_vocabulary_pd = puzzle_vocabulary_pd.join(fen_data.set_index('tactics_problem_id'))

In [146]:
#IMPORTANT remove empty values, causes issues with projector
puzzle_vocabulary_pd = puzzle_vocabulary_pd.replace(np.nan, 0,regex=True)
puzzle_vocabulary_pd.reset_index(level=0, inplace=True)

In [147]:
#Set data types back to int32 for numeric columns
puzzle_vocabulary_pd.astype({'rating': 'int32','attempt_count': 'int32','average_seconds':'int32','move_count':'int32'}).dtypes
#Check for Null values
print(len(puzzle_vocabulary_pd) - puzzle_vocabulary_pd.count())
puzzle_vocabulary_pd.head()

index                 0
tactics_problem_id    0
rating                0
rd                    0
attempt_count         0
average_seconds       0
move_count            0
tags                  0
fen                   0
dtype: int64


,index,tactics_problem_id,rating,rd,attempt_count,average_seconds,move_count,tags,fen
0,0,731,1569.0,44.96,53336.0,41.0,3.0,"Attacking f7/f2,Attacking Castled King,Back Ra...",https://lichess.org/analysis/r1b2rk1/ppp2ppp/1...
1,1,30293,691.0,43.45,46324.0,34.0,1.0,0,https://lichess.org/analysis/2k5/pp3ppp/8/2p5/...
2,2,765,588.0,42.31,45728.0,18.0,1.0,Mate in 1,https://lichess.org/analysis/r1bqk2r/pppn1pp1/...
3,3,29166,629.0,41.37,42273.0,48.0,2.0,"Mate in 3+,Queen Sacrifice",https://lichess.org/analysis/4nrk1/3R1ppp/6q1/...
4,4,29064,986.0,43.76,57201.0,45.0,2.0,"Basic Checkmates,Mating Net,Sacrifice",https://lichess.org/analysis/8/4Q3/k1q5/5p2/r2...


<h2>Save Files for Projector</h2>

In [148]:
#Save Metadata
puzzle_vocabulary_pd.to_csv(
        'meta.tsv',
        sep='\t',
        index=False,
        header=True,
        encoding='utf-8'
    )

In [118]:
#Save vectors
vectors.to_csv(
        'vectors.tsv',
        sep='\t',
        index=False,
        header=False,
        encoding='utf-8'
    )

In [152]:
#Generate Config For Projector
import json
embeddings = []
embeddings.append({"tensorName": "Chess Puzzle Embeddings 1",
      "tensorShape": [
        vectors.shape[0],
        vectors.shape[1]
      ],
      "tensorPath": "https://raw.githubusercontent.com/biltmire/ChessPuzzleEmbeddings/master/dim200vectors.tsv",
      "metadataPath": "https://raw.githubusercontent.com/biltmire/ChessPuzzleEmbeddings/master/dim200meta.tsv",
  })

with open('../../ChessPuzzleEmbeddings/puzzle_config.json', "w+") as text_file:
    text_file.write(json.dumps({"embeddings": embeddings}))